In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.HuggingFace, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Qdrant, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc4 Microsoft.SemanticKernel.Connectors.HuggingFace, 1.0.0-rc4 Microsoft.SemanticKernel.Connectors.Qdrant, 1.0.0-rc4

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.Qdrant;
using Microsoft.SemanticKernel.Connectors.HuggingFace;

In [3]:
string chat_endpoint = "http://localhost:5002/v1/chat/completions";
string embeddings_endpoint = "http://localhost:5002/v1/embeddings";
string qdrant_endpoint = "http://localhost:6333";

In [5]:

#pragma warning disable SKEXP0020

Kernel kernel = Kernel.CreateBuilder()
            .AddHuggingFaceTextGeneration(
                model: "phi15",
                endpoint: chat_endpoint)
            .Build();

In [9]:
#pragma warning disable SKEXP0052
#pragma warning disable CS1061
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0026

#pragma warning disable SKEXP0020

var qdrantMemoryBuilder = new MemoryBuilder();

var hfembeddings = new HuggingFaceTextEmbeddingGenerationService("jina", embeddings_endpoint);

qdrantMemoryBuilder.WithTextEmbeddingGeneration(hfembeddings);
qdrantMemoryBuilder.WithQdrantMemoryStore(qdrant_endpoint, 768);

var builder = qdrantMemoryBuilder.Build();

In [10]:
string MemoryCollectionName = "demoenvb";

await builder.SaveInformationAsync(MemoryCollectionName, id: "id1", text: "kinfey is cloud advocate");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id2", text: "kinfey is Microsoft MVP");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id3", text: "kinfey is AI expert");
await builder.SaveInformationAsync(MemoryCollectionName, id: "id4", text: "Satya is Microsoft CEO");


In [11]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "who is kinfey", limit: 3, minRelevanceScore: 0.6);

In [12]:
await foreach (var item in searchResults)
{
    Console.WriteLine(item.Metadata.Text + " : " + item.Relevance);
}

kinfey is AI expert : 0.8911192
kinfey is Microsoft MVP : 0.87062556
kinfey is cloud advocate : 0.84311897


In [6]:
var questionAnswerFunction = kernel.CreateFunctionFromPrompt("ask: {{$input}} answer:");

In [7]:
var result = await kernel.InvokeAsync(questionAnswerFunction, new(){["input"] = "who are you"});


In [8]:
result.GetValue<string>()

ask: who are you answer:

- I am a human, a living being that breathes, eats, and grows. I have a brain, a part of my body that helps me think, learn, and remember. I have a heart, a part of my body that pumps blood, which carries oxygen and nutrients to my cells. I have a stomach, a part of my body that digests food, which is the process of breaking down food into smaller pieces and absorbing the nutrients. I have a tongue, a part of my body that helps me taste, which is the sense of what something is like. I have a nose, a part of my body that helps me smell, which is the sense of what something smells like. I have a mouth, a part of my body that helps me speak, which is the way I communicate with others. I have a hair, a part of my body that helps me protect my skin, which is the outer layer of my body